In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __builtin__ import any as b_any

import math
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""
import numpy as np
from PIL import Image
from alexnet import AlexNet

import tensorflow as tf

import configuration
import inference_wrapper
import sys
sys.path.insert(0, 'im2txt/inference_utils')
import caption_generator
import vocabulary

In [2]:
np.random.seed(2)

current_dir = os.getcwd()
image_dir = os.path.join(current_dir, 'Jmemes')


checkpoint_path="trainlogIncNEW"
vocab_file ="vocab4.txt"
input_files ="Jmemes"         

tf.logging.set_verbosity(tf.logging.INFO)


In [3]:
#configuration
config = configuration.ModelConfig()

g = tf.Graph()
with g.as_default():
    model = inference_wrapper.InferenceWrapper()
    restore_fn = model.build_graph_from_config(config,
                                               checkpoint_path)
g.finalize()


INFO:tensorflow:Building model.
BUILDING DENSE
About to decide if splitting
new_h Tensor("lstm/basic_lstm_cell/Mul_2:0", shape=(1, 512), dtype=float32)
new_state LSTMStateTuple(c=<tf.Tensor 'lstm/basic_lstm_cell/Add_1:0' shape=(1, 512) dtype=float32>, h=<tf.Tensor 'lstm/basic_lstm_cell/Mul_2:0' shape=(1, 512) dtype=float32>)
About to decide if splitting
It's splitting
new_h Tensor("lstm/basic_lstm_cell/Mul_5:0", shape=(1, 2, 512), dtype=float32)
new_state LSTMStateTuple(c=<tf.Tensor 'lstm/basic_lstm_cell/Add_3:0' shape=(1, 2, 512) dtype=float32>, h=<tf.Tensor 'lstm/basic_lstm_cell/Mul_5:0' shape=(1, 2, 512) dtype=float32>)
lstm_outputs Tensor("lstm/basic_lstm_cell/Mul_5:0", shape=(1, 2, 512), dtype=float32)
state_tuple LSTMStateTuple(c=<tf.Tensor 'lstm/basic_lstm_cell/Add_3:0' shape=(1, 2, 512) dtype=float32>, h=<tf.Tensor 'lstm/basic_lstm_cell/Mul_5:0' shape=(1, 2, 512) dtype=float32>)


In [4]:
vocab = vocabulary.Vocabulary(vocab_file)

with open('Captions.txt','r') as f:
    data_captions = f.readlines()
data_captions = [s.lower() for s in data_captions]
  
with open('ordered_memes.txt','r') as f:
    ordered_memes = f.readlines()
ordered_memes = [meme.replace('\n','') for meme in ordered_memes]

INFO:tensorflow:Initializing vocabulary from file: vocab4.txt
INFO:tensorflow:Created vocabulary with 38521 words


In [5]:
np.random.seed(2)
#convert jpg image(s) into iamge representations using alexnet:
filenames = [os.path.join(image_dir, f) for f in os.listdir(input_files) if f.endswith('.jpg')]
#filenames = [os.path.join(image_dir, f) for f in ordered_memes[150:160] + ['TutorPP.jpg']]
print(filenames)
tf.logging.info("Running caption generation on %d files matching %s",len(filenames), input_files)
with tf.Session(graph=g) as sess:
    # Load the model from checkpoint.
    restore_fn(sess)
    
    # Prepare the caption generator. Here we are implicitly using the default
    # beam search parameters. See caption_generator.py for a description of the
    # available beam search parameters.
    generator = caption_generator.CaptionGenerator(model, vocab, beam_size=config.beam_size)
    num_in_data_total = 0
    num_captions = 0
    for i,filename in enumerate(filenames):
      with tf.gfile.GFile(filename, "rb") as f:
        image = f.read()
      print("Captions for image %s:" % os.path.basename(filenames[i]))
      num_in_data = 0
      for k in range(10):
          captions = generator.beam_search(sess, image)
            
          for i, caption in enumerate(captions):
            # Ignore begin and end words.
            sentence = [vocab.id_to_word(w) for w in caption.sentence[1:-1]]
            sentence = " ".join(sentence)
            in_data = 0
            if b_any(sentence in capt for capt in data_captions):
                in_data = 1
                num_in_data += 1
                num_in_data_total += 1
                num_captions += 1
            else:
                num_captions += 1
            print("  %d) %s (p=%f) [in data = %d]" % (i, sentence, math.exp(caption.logprob),in_data))
      print("number of captions in data = %d" % (num_in_data))
    print("(total number of captions in data = %d) percent in data = %f" % (num_in_data_total,(num_in_data_total/num_captions)))

['/Users/freedmand/scraps/danklearning/im2txt/Jmemes/sassy-gay-snape.jpg', '/Users/freedmand/scraps/danklearning/im2txt/Jmemes/romneyhood.jpg', '/Users/freedmand/scraps/danklearning/im2txt/Jmemes/ridiculously-photogenic-metalhead-guy.jpg', '/Users/freedmand/scraps/danklearning/im2txt/Jmemes/sad-dog.jpg', '/Users/freedmand/scraps/danklearning/im2txt/Jmemes/sudden-realization-ralph.jpg', '/Users/freedmand/scraps/danklearning/im2txt/Jmemes/stoner-stanley.jpg', '/Users/freedmand/scraps/danklearning/im2txt/Jmemes/stoner-dog.jpg', '/Users/freedmand/scraps/danklearning/im2txt/Jmemes/advice-dog.jpg']
INFO:tensorflow:Running caption generation on 8 files matching Jmemes
INFO:tensorflow:Loading model from checkpoint: trainlogIncNEW/model.ckpt
INFO:tensorflow:Restoring parameters from trainlogIncNEW/model.ckpt
INFO:tensorflow:Successfully loaded checkpoint: model.ckpt
Captions for image sassy-gay-snape.jpg:
  0) my brother (p=0.000005) [in data = 1]
  1) my brother called me a witch (p=0.000000) 

  0) the first supper is an excuse for the summer is . . . (p=0.000000) [in data = 0]
  1) the first supper is an excuse for the summer is . . . for you (p=0.000000) [in data = 0]
number of captions in data = 0
Captions for image stoner-stanley.jpg:
  0) hi i'm carl (p=0.000003) [in data = 1]
  1) hi i'm carl and im a unicorn (p=0.000000) [in data = 0]
  0) dude and i was like shit (p=0.000000) [in data = 0]
  1) dude and i was like a big bird ! (p=0.000000) [in data = 0]
  0) the face you realize (p=0.000001) [in data = 0]
  1) the face you realize when you forgot to tell me (p=0.000000) [in data = 0]
  0) you like turtles ! (p=0.000000) [in data = 0]
  1) you can call me mr . you (p=0.000000) [in data = 0]
  0) my names dave . . . . . . . (p=0.000000) [in data = 0]
  1) my names dave . . . . . . i just bought my burger (p=0.000000) [in data = 0]
  0) i want to talk to your little girls (p=0.000000) [in data = 0]
  1) i want to talk to your little sister to me (p=0.000000) [in data = 

In [6]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1000px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [7]:
show_graph(g)

In [8]:
OLD_CHECKPOINT_FILE = "trainlogSGD/model.ckpt-600000"
NEW_CHECKPOINT_FILE = "trainlogSGDNEW/model.ckpt-600000"

import tensorflow as tf
vars_to_rename = {
    "lstm/basic_lstm_cell/weights": "lstm/basic_lstm_cell/kernel",
    "lstm/basic_lstm_cell/biases": "lstm/basic_lstm_cell/bias",
}
new_checkpoint_vars = {}
reader = tf.train.NewCheckpointReader(OLD_CHECKPOINT_FILE)
for old_name in reader.get_variable_to_shape_map():
  if old_name in vars_to_rename:
    new_name = vars_to_rename[old_name]
  else:
    new_name = old_name
  new_checkpoint_vars[new_name] = tf.Variable(reader.get_tensor(old_name))

init = tf.global_variables_initializer()
saver = tf.train.Saver(new_checkpoint_vars)

with tf.Session() as sess:
  sess.run(init)
  saver.save(sess, NEW_CHECKPOINT_FILE)

InvalidArgumentError: Unsuccessful TensorSliceReader constructor: Failed to get matching files on trainlogSGD/model.ckpt-600000: Not found: trainlogSGD; No such file or directory